## C++ en confinement présente les *templates*

Nous allons parler de modèles de classe. C++ parlant anglais,
comme tous les langages de programmation, nous allons parler de
*templates*.

Vous allez vous dire que nous sommes vraiment des maniaques des
notebooks et qui plus est là ne serviront même pas à exécuter le
code. Alors déjà c'est possible d'exécuter du c++ dans des notebooks (pas dans ceux-ci 
mais avec xeus et le compilateur cling) mais ce n'est pas aussi bien que
pour le python, aussi nous n'allons surtout pas vous demander
d'installer quoi que ce soit, les notebooks vont simplement nous
servir à vous donner des explications suivies de (jolis) morceaux de code que
vous pourrez copier/coller dans vos éditeurs (VSC) et compiler dans votre terminal. Nous en entondons qui disent
que c'est vraiment nul, ils ont raison mais en période de confinement
on fait ce qu'on peut avec ce qu'on a ...

allez c'est parti pour *tout ce que vous auriez voulu savoir sur les templates*


Voilà donc vous connaissez désormais un petit peu c++ et vous savez
que c'est un langage typé, contrairement à Python où il n'est pas
nécessaire de définir le type d'un objet pour pouvoir l'utiliser.

Supposons qu'un client (un peu comme ceux de vos projets informatique
mais en beaucoup plus pénible) vous demande d'implémenter une fonction
pour diviser deux entiers. Vous écrivez le superbe code suivant (que vous pouvez naturellement essayer d'implémenter par vous-même)


```c++
int my_div (const int & a , const int & b ) {
  return a/b ;
}
```

Alors là vous voyez des `const` et bien oui la fonction `my_div` n'a pas besoin de modifier ses arguments, donc nous les mettons en lecture seule. Disons que c'est surtout pour nous éviter de les mofidier par mégarde ... c'est une aide à une programmation plus sûre

Vous voyez aussi des références `&` et bien oui vous passez vos deux entiers par reférence afin d'éviter à c++ de les copier ... vous vous dites que c'est un peu stupide pour un entier (et que de plus on crée une *indirection* puisqu'on travaille sur des adresses, plutôt que de copier deux entiers) ... mais réfléchissez, nous voulons faire du code "générique", pensons à un autre type qu'entier, par exemple deux (super grosses) matrices (ok la division de deux matrices est un non-sens mathématique mais supposons que ca existe), donc si on passe les deux matrices par copie à la fonction (c'est à dire sans le `&`) la fonction va recopier complètement en mémoire les matrices et tout cela pour simplement les regardez !  là c'est un non sens-informatique...

et vous le testez votre code

```c++
#include <iostream>

int main () {
  std::cout << my_div(10, 3) << std::endl;
  return 0;
}
```

La fonction marche super bien vous êtes très content. Mais le client
revient et vous dit qu'il veut aussi diviser deux doubles. int double ce sont des types numériques, essayons déjà ce qu'on possède

```c++
int main () {
   std::cout << div(10., 3.) << std::endl;
   return 0;
}
```

heu... votre fonction fait une division entière c'est pas bon alors
vous faite une nouvelle fonction div qui prend deux doubles et qui
rend un double.

```c++
double my_div (const double & a, const double & b ) {
  return a/b ;
}
```

Au passage vous remarquez que c++ accepte dans un même code des
fonctions qui ont le même nom puisqu'il arrive à les discriminer par
leurs arguments (nombre ou type, ici type) c'est ce qu'on appelle l'overloading

Maintenant votre client revient il veut diviser des short, bon ça
commence à bien faire, vous en avez assez de ce code super redondant
(et de ce client aussi) donc vous respirez un grand coup et vous
remarquez que le code n'est pas si différent voire franchement
semblable d'une fonction div à une autre. Et comme tout le monde en
programmation, vous êtes super paresseux, vous détestez écrire
plusieurs fois la même chose (surtout que vous utilisez le
copier/coller et que vous faites des tas d'erreurs stupides qui prennent un temps fou à corriger, et oui c'est classique en programmation).

Et vous pressentez déjà que votre client n'en restera pas là... vous
vous dites que dans un langage de programmation aussi magnifique que
c++, il doit bien exister un mécanisme dédié pour éviter ce genre de
redondance de code !

La réponse est que ca existe ! Au moment de l'implémentation d'un
code, vous allez dire qu'une partie d'un code, que ce soit une
fonction ou une classe, sera utilisable pour n'importe quel type de
données. Et au moment de la compilation c++ se chargera de générer
automatiquement toutes les fonctions nécessaires. Il
le fera bien sûr à partir des modèles que vous lui avez indiqués. Et
il ne mettra naturellement que ce qui est utile ! c'est top

Voilà vous avez compris l'idée de base des templates: vous allez
écrire du code que le compilateur va utiliser pour produire un autre
code c'est pour cela que les gens appelle ça de la méta-programmation.

```c++
#include <iostream>

template < typename T>
T my_div (const T & a, const T & b ) {
  return a/b ;
}
int main () {
  std::cout << my_div(10, 3) << std::endl;
  std::cout << my_div(10., 3.) << std::endl;
  return 0;
}
```

Vous remarquez plusieurs choses: (i) que nous déclarons que la fonction que nous implémentons est un modèle (template) de fonction; (ii) que le nom choisi pour le type est T (là on aime mettre des majuscules) et (iii) que votre fonction consiste à prendre en argument deux références (&) vers des objets constants (const) de type T et à renvoyer leur division (/). Alors là déjà vous remarquez un premier problème potentiel: si pour le type T l'opérateur / n'est pas défini, vous allez avoir une erreur... on essaie:

``` c++
// file tmpl_div.cpp
template < typename T>
T my_div (const T a, const T b ) {
  return a/b ;
}
class X {}; // pas de redéfinition de operator/ dans la classe X
int main () {
  X x1, x2;
  X x3 = my_div(x1, x2);
  return 0;
}
```
```
$ g++ tmpl_div.cpp
tmpl_div.cpp: In instantiation of ‘T my_div(T, T) [with T = X]’:
tmpl_div.cpp:10:23:   required from here
tmpl_div.cpp:5:11: error: no match for ‘operator/’ (operand types are ‘const X’ and ‘const X’)
   return a/b ;
          ~^~
```

ok c++ refuse de compiler ce programme en faisant une erreur au moment de la compilation (décidément pas facile de le tromper celui là). Ce mécanisme de génération de code à la compilation vous paraît vraiment extrêmement
puissant et vous avez raison, il est responsable en grande partie du grand succès de c++ et vous n'avez encore rien vu, attendez de voir le *pretty print* de Basile mais plus tard...  restons encore un moment sur les bases


Mais voilà que votre client revient à nouveau (vous pensez fugacement à changer de domaine, peut être les matériaux) avec une nouvelle demande: une fonction add pour additionner des entiers, des doubles mais aussi maintenant des chaînes de caractères (ca doit les concaténer) alors vous réflechissez déjà c'est facile si il a des chaînes de caractères sous la forme de std::string, puisqu'elles ont l'opérateur +

```c++
// file tmpl_add.cpp
#include <iostream>
#include <string>
template < typename T>
T my_add (const T a, const T b ) {
  return a+b ;
}
int main () {
   std::cout << my_add(12, 4) << std::endl;
   std::cout << my_add(12.7, 4.92) << std::endl;
   std::string s1 ("on");
   std::string s2 (" confine");
   std::cout << my_add(s1, s2) << std::endl;
      ;
    return 0;
}
```
```
$ g++ tmpl_add.cpp -o tmpl_add
$ tmpl_add
16
17.62
on confine
```

pas de chance, votre client voulait des chaînes de caractères *c-like* des années 80 sous la forme de char* (décidément ce client qui ne connait même pas l'informatique de base...) une seule fonction template ne suffit plus, les char* n'ont pas de fonction '+'. Qu'à cela ne tienne, c++ vous permet de spécialiser des fonctions templates i.e. les redéfinir pour certains types de données.

```c++
template < typename T>
T my_add (const T a, const T b) {
  return a+b ;
}

char* my_add (const char* a, const char* b) {
  return "add on char* pas intéressant à implémenter: préfèrez les std::string !" ;
}

int main () {
  std::cout << my_add(10, 3) << std::endl;
  const char* s1 = "on";
  const char* s2 = " confine";
  std::cout << my_add(s1, s2);
  return 0;
}
```
```
$ g++ my_add.cpp
(là un petit warning)
"add on char* pas intéressant à implémenter: préfèrez les std::string !" ;
```

bon très bien tout cela, mais on vous promet un mécanisme super sophistiqué et pour l'instant on vous a fait coder de petites opérations... pas extraordinaire tout ça. Et vous avez raison, les classes aussi peuvent être des modèles de classes. Commencons par jouer avec un tout petit exemple: une classe Toy, composée d'un élément de type T et sur laquelle on définit la fonction play qui affiche un message et l'élément et d'une autre fonction tidy qui fait de même.

```c++
#include <iostream>
template <typename T>
class Toy {
private:
  T element;
public:
  Toy (T e): element(e) {}
    
  void play () {
    std::cout << "Playing with " << element << std::endl;
  }
    
  void tidy () {
    std::cout << "Je range " << element << std::endl;
  }
};
```

et le main
```c++
Toy<float> feu(1.7);
int main () {
  feu.play();
  Toy<const char*> jeu("c++");
  jeu.play();
  Toy<int>* peu = new Toy<int>(12);
  peu->tidy();
  Toy<bool>* beu;
}
```

et on compile
```
$ g++ toy.cpp -o toy
$ ./toy
Playing with 1.7
Playing with c++
Je range 12

```

Vous remarquez des tas de choses...

Déjà la classe Toy est une classe template, le type non défini s'appelle (typename) T, dans la définition de la classe Toy, T apparaît comme le type du membre de donné element et comme le type de l'argument passé au constructeur.

Dans la partie main c'est encore plus intéressant ! vous voyez qu'on demande la création d'une variable globale feu de type Toy ou le type de element est un float (avec $<$float$>$) et qu'on l'initialise en passant la valeur 1.7 à son constructeur, dans le main on crée une variable locale jeu de type Toy où T est const char* avec (avec <const char*>) et ensuite une variable peu qui est un pointeur de type Toy où T est int qui pointe vers un objet aloué en mémoire dynamique, et qu'on définit un pointeur vers une variable de type Toy où T est bool. Vous constatez que les appels des fonctions sont classiques.

Intéressons nous à la compilation d'un tel code. Quelles sont les fonctions que c++ va générer à partir de ce code ?

Il va générer toutes les fonctions dont vos objets ont besoin (qu'ils les utilisent utilisent explicitement ou implicitement - vous voyez où je veux en venir).

Quels sont les constructeurs que c++ genère ? oui Toy$<$float$>$::Toy(float) {...} (le constructeur de la classe Toy où T est float), oui aussi Toy$<$const char\*$>$::Toy(const char\*){...} et oui aussi Toy$<$int$>$::Toy(int){...} ... et pour beu ? et bien non rien c'est un pointeur non initialisé (ce qui est très dangereux) vers un objet de type Toy$<$bool$>$ mais l'objet n'a pas été construit donc pas besoin du constructeur pour les Toy *booléans* ...

pour les fonctions ? pareil il génère celles qui sont appelées donc Toy$<$float$>$::play() {...}, Toy$<$const char\*$>$::play() {...} et enfin Toy$<$int$>$::tidy().


Pensez-vous que ce soit tout ? (si on demande vous savez donc que ce n'est pas tout ...) non bien sûr, rappelez-vous, quand un objet arrive en fin de vie, que lui arrive-t-il ? il est détruit, donc c++ doit générer les desctructeurs aussi mais lesquels et bien logiquement les types pour lesquels il a généré le constructeur, donc pas pour les Toy *booléens* !


Voila, vous avez désormais compris: ce mécanisme de template complètement différent d'une compilation classique il intervient avant
la compilation classique et il permet de construire et compiler votre code à la
volée.

Vous avez écrit un code qui permet de générer un autre code: vous avez fait de la meta programmation. C'est cool.

Nous en entendons déjà les pragmatiques qui disent *mais si je veux définir mes fonctions en dehors de la classe comment je dois faire ?* Ok nous leur donnons la syntaxe...

```c++
#include <iostream>
template <typename T>
class Toy {
private:
  T element;
public:
  Toy (T e);
  void play ();
  void tidy ();
};

template <typename T>
inline Toy<T>::Toy (T e): element(e) {}

template <typename T>
inline void Toy<T>::play () {
  std::cout << "Playing with " << element << std::endl;
}
    
template <typename T>
inline void Toy<T>::tidy () {
  std::cout << "Je range " << element << std::endl;
}

int main () {
  Toy<const char*> j("c++");
  j.play();
}
```

Alors là l'un d'entre vous remarque tout de suite le mot clé inline qui a hanté vos cours de c++ ! alors on explique. Pour faire court, les fonctions définies (i.e. dont on explicite le corps) à l'intérieur de la classe (dans les { }) sont des fonctions inline par défaut mais les fonctions définies à l'extérieur de la classe (à l'extérieur des {}) ne le sont pas, il faut le préciser pour qu'elles le soient.

et inline dans tout cela ? ah oui, ca indique simplement au compilateur qu'il doit
remplacer les appels de fonctions par le corps des fonctions et ainsi
plus de perte de temps inutile lors de l'appel de la fonction: on ne va pas chercher son code en mémoire (elle n'a pas de code donc pas d'adresse), on n'empile pas ses arguments sur la pile d'exécution de c++, on en dépile pas non plus ses arguments une fois la fonction terminée.



Maintenant à vous de jouer !

Reprenons donc le code de notre célèbre
intstack. C'est un conteneur qui (par pur hasard) contient des entiers
mais naturellement son mécanisme serait le même pour des chaînes de
caractères, de Matrices...

Vous l'avez compris, la solution qui consisterait à faire une classe
StringStack coupant/collant le code de la IntStack et en
remplacant (aux bons endroits) les int par des std::string (maintenant que vous connaissez les templates) n'est pas la bonne solution... mais disons que c'est néanmoins comme cela qu'il faut commencer vos templates (même B. Stroustrup le dit): vous commencez par implémenter vos templates sur un type existant (genre int comme votre IntStack), et une fois votre code bien débuggé et bien testé, vous le généralisez en remplacant le type choisi par T ! 

Donc vous allez prendre votre classe IntStack, dire qu'elle est template (i.e. elle manipule un ou plusieurs types non spécifiés) et remplacer les int par un paramètre T (aux bons endroits ... heu non pas le int indiquant le type de la taille de la pile ... non plus le int du type de top ... oui celui qui est passé en argument de push ...).

Comme en période de confinement, de pales et de campagne BDE,
les codes des IntStack sont super difficiles à retrouver, nous vous en
donnons un au cas où (mais naturellement préférez le vôtre). Attention: peut-il y avoir des erreurs dans ce code ? mais oui bien sûr ! soyez toujours méfiant devant le code des autres surtout quand il vient sans son jeu de tests... Il y a bien une erreur dans la fonction  operator= (saurez-vous la trouver ? sinon gare aux segfaults quand vous affecterez un objet de type IntStack à un autre objet de type IntStack existant). En période de confinement on s'amuse comme on peut...

Au passage nous pensons que vous en avez super marre de copier/coller la définition de la classe IntStack dans des fichiers .cpp aussi nous avons naturellement mis la définition de la IntStack dans un fichier de header (un .h) qui peut être inclus dans un autre fichier (soir .h soit .cpp) et nous l'avons protégé contre plusieurs inclusions de ce même fichier lors d'une compilation. Pour faire court au premier #include "intstack.h" rencontré lors d'une compilation, la variable shell INTSTACK_H n'existe pas (i.e. #ifndef INTSTACK_H est true et que ifndef = if-not-defined), on la définit (i.e. #define INTSTACK_H est exécuté), maintenant dans ce même fil de compilation quand on va repasser sur une inclusion de intstack.h, la variable INTSTACK_H existera et on ne passera pas dans le corps de la branche then du ifndef qui définit la classe.

```c++
// dans le fichier intstack.h
#ifndef INTSTACK_H
#define INTSTACK_H

#include <iostream>
#include <stdexcept>

class IntStack {
private:
  int * tab;
  int top;
  int size;

public:

  IntStack (int s): size(s), top(0) {
    if (size <= 0)
      throw std::range_error("Bad stack size");
    else
      tab = new int[s];
  }

  IntStack (const IntStack& rs): size(rs.size), top(rs.top) {
    tab = new int[size];
    for (int i=0; i < top ; ++i)
      tab[i] = rs.tab[i];
  }

  ~IntStack(){
    delete [] tab;
  }

  IntStack& operator= (const IntStack& rs) {
    if (this != &rs){
      delete [] tab;
      tab = new int[size];
      top = rs.top;
      for (int i=0; i<top ; ++i){
        tab[i] = rs.tab[i];
      }
    }
    return * this;
  }

  bool is_empty () const {
    return top == 0;
  }
  
  bool is_full () {
    return top == size;
  }

  void push (int e) {
    if (is_full())
      throw std::range_error("The stack is full");
    tab[top] = e ;
    top = top + 1;
  }

  void print (std::ostream& os) const {
    os << "[ ";
    for (int i=0 ; i<top; ++i)
      os << tab[i] << ' ';
    os << "[" << std::endl;
  }

  int pop() {
    if (is_empty())
      throw std::range_error("The stack is empty");
    top = top - 1;
    return tab[top];
  }
};
#endif // INTSTACK_H
```

Prenez le temps de programmer votre stack template ... (dessous ce sont de petites remarques)

Vous remarquez que: la construction d'une pile de taille négative ou nulle, le dépilement d'une pile vide et l'empilement sur une pile pleine déclenchent des erreurs c++ (throw) et que nous avons choisi de prendre des erreurs pré-définies par c++ dans stdexcept là http://www.cplusplus.com/reference/stdexcept/ vous pouvez naturellement en choisir d'autres chacun ses goûts... voire faire les vôtres (mais nous on applique L'axiome de la programmation *moins on en fait mieux on se porte* et on récupère tout ce qu'on peut dans les endroits en lesquels on a confiance (oui genre la lib std de c++)

Quelqu'un a-t-il remarqué les deux const qui apparaissent dans de drôles d'endroits dans les fonctions is\_empty() et print(). Quelqu'un a-t-il une idée de l'utilité ?

Et bien je ne vais pas vous répondre directement mais vous allez essayer de compiler les deux codes suivants et vous en déduirez de vous même à quoi servent ces const... (parce qu'il en manque un et que ca va faire une erreur dans le second code).

```c++
#include <iostream>
#include "intstack.h"
void foo (const IntStack& ri) {
  if (ri.is_empty())
    std::cout << "la pile d'entiers passée à foo est vide !";
  else
    ri.print(std::cout);
}
	  
int main () {
  IntStack s (3);
  foo(s);
  return 0;
}
```
```
$ g++ intstack.cpp -o intstack
$ ./intstack
la pile d'entiers passée à foo est vide !
```

pas de problème le suivant maintenant ...

```c++
#include <iostream>
#include "intstack.h"

void bar (const IntStack& ri) {
  if (ri.is_full())
    std::cout << "la pile d'entiers passée à foo est pleine !";
  else
    ri.print(std::cout);
}
	  
int main () {
  IntStack s (3);
  foo(s);
  return 0;
}

```

Avez-vous compris où le const manque et à quoi il sert ?

END